In [1]:
#imports
import pandas as pd

In [2]:
#load original dfs
original_df = pd.read_csv("../data/aggregated_data.csv")

In [19]:
#methods to create change_by_year data

def generateChangeDataFeatureColumns(df, column):
    df[f'{column}_val_change'] = None 
    df[f'{column}_perc_change'] = None 
    # Create an empty DataFrame with the same columns as the original one 
    new_df = pd.DataFrame(columns=df.columns) 
    # Get unique states and years 
    states = df["state"].unique() 
    years = df["YEAR"].unique()
    # Iterate over states and years 
    for state in states: 
        for year in years:
            print(year - 1 in years) 
            if (year - 1) in years: 
                row_index = df[(df['state'] == state) & (df['YEAR'] == year)].index
                last_years_row = df[(df['state'] == state) & (df['YEAR'] == year - 1)] 
                if not row_index.empty and not last_years_row.empty: 
                    current_value = df.at[row_index[0], column]
                    last_value = last_years_row[column].values[0] 
                    # Calculate the value change and percentage change 
                    df.at[row_index[0], f'{column}_val_change'] = current_value - last_value 
                    df.at[row_index[0], f'{column}_perc_change'] = (current_value- last_value) / last_value
                # else: print(f"No data found for State: {state}, Year: {year} or Year: {year - 1}") 
            else: 
                print(f'No previous year found for Year: {year} in State: {state}')

    return df

def generateChangeDataTargetColumns(df, column):
    df[f'{column}_val_change'] = None 
    df[f'{column}_perc_change'] = None 
    # Create an empty DataFrame with the same columns as the original one 
    new_df = pd.DataFrame(columns=df.columns) 
    # Get unique states and years 
    states = df["state"].unique() 
    years = df["YEAR"].unique() 
    # Iterate over states and years 
    for state in states: 
        for year in years: 
            if year + 1 in years:
                row_index = df[(df['state'] == state) & (df['YEAR'] == year)].index
                next_years_row = df[(df['state'] == state) & (df['YEAR'] == year + 1)] 
                if not row_index.empty and not next_years_row.empty: 
                    current_value = df.at[row_index[0], column]
                    next_value = next_years_row[column].values[0] 
                    # Calculate the value change and percentage change 
                    df.at[row_index[0], f'{column}_val_change'] = next_value - current_value 
                    df.at[row_index[0], f'{column}_perc_change'] = (next_value- current_value) / current_value
                else: print(f"No data found for State: {state}, Year: {year} or Year: {year + 1}") 
            else: 
                print(f'No next year found for Year: {year} in State: {state}')  
    return df  



In [20]:
change_df = original_df
feature_cols = ["mean_ADHD", "mean_PTSD", "mean_anxiety", "mean_bipolar", "mean_depression",
                "mean_mental hospital", "mean_psychologist near me", "mean_psychiatrists near me", "mean_therapist near me",
                "median_ADHD", "median_PTSD", "median_anxiety", "median_bipolar", "median_depression",
                "median_mental hospital", "median_psychologist near me", "median_psychiatrists near me", "median_therapist near me"
                ]

for col in feature_cols:
    change_df = generateChangeDataFeatureColumns(change_df, col)
print(len(change_df))
change_df = generateChangeDataTargetColumns(change_df, "UTIL_RATE")


change_df.to_csv('../data/aggregated_data_with_changes.csv', index=False)


True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: AL
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: AK
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: AZ
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: AR
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: CA
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: CO
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: CT
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: DE
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: DC
True
True
True
True
True
True
True
True
True
False
No previous year found for Year

C:\Users\johnb\AppData\Local\Temp\ipykernel_32376\1074925728.py:23: RuntimeWarning: invalid value encountered in scalar divide
  df.at[row_index[0], f'{column}_perc_change'] = (current_value- last_value) / last_value
C:\Users\johnb\AppData\Local\Temp\ipykernel_32376\1074925728.py:23: RuntimeWarning: divide by zero encountered in scalar divide
  df.at[row_index[0], f'{column}_perc_change'] = (current_value- last_value) / last_value


True
True
True
True
False
No previous year found for Year: 2013 in State: MA
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: MI
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: MN
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: MS
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: MO
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: MT
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: NE
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: NV
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: NJ
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: NM
True


C:\Users\johnb\AppData\Local\Temp\ipykernel_32376\1074925728.py:23: RuntimeWarning: divide by zero encountered in scalar divide
  df.at[row_index[0], f'{column}_perc_change'] = (current_value- last_value) / last_value
C:\Users\johnb\AppData\Local\Temp\ipykernel_32376\1074925728.py:23: RuntimeWarning: invalid value encountered in scalar divide
  df.at[row_index[0], f'{column}_perc_change'] = (current_value- last_value) / last_value


True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: HI
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: ID
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: IL
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: IN
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: IA
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: KS
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: KY
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: LA
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: MA
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in 

...

True
True
True
False
No previous year found for Year: 2013 in State: AK
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: AZ
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: AR
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: CA
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: CO
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: CT
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: DE
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: DC
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: FL
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: GA
True
True


C:\Users\johnb\AppData\Local\Temp\ipykernel_32376\1074925728.py:23: RuntimeWarning: invalid value encountered in scalar divide
  df.at[row_index[0], f'{column}_perc_change'] = (current_value- last_value) / last_value
C:\Users\johnb\AppData\Local\Temp\ipykernel_32376\1074925728.py:23: RuntimeWarning: divide by zero encountered in scalar divide
  df.at[row_index[0], f'{column}_perc_change'] = (current_value- last_value) / last_value


True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: KY
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: LA
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: MA
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: MI
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: MN
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: MS
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: MO
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: MT
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: NE
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in 

C:\Users\johnb\AppData\Local\Temp\ipykernel_32376\1074925728.py:23: RuntimeWarning: divide by zero encountered in scalar divide
  df.at[row_index[0], f'{column}_perc_change'] = (current_value- last_value) / last_value
C:\Users\johnb\AppData\Local\Temp\ipykernel_32376\1074925728.py:23: RuntimeWarning: invalid value encountered in scalar divide
  df.at[row_index[0], f'{column}_perc_change'] = (current_value- last_value) / last_value


True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: NC
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: ND
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: OH
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: OK
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: OR
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: PA
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: RI
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: SC
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 2013 in State: SD
True
True
True
True
True
True
True
True
True
False
No previous year found for Year: 201

In [93]:
row = test_df[(test_df['STATE'] == 'CT') & (test_df['YEAR'] == 2018)]
print(test_df['mean_ADHD_perc_change'])
val_change = row['mean_ADHD_val_change'].values[0] 
perc_change = row['mean_ADHD_perc_change'].values[0] 
val_2018 = row["mean_ADHD"].values[0]
print(f"Value Change for AL in 2018: {val_change}") 
print(f"Percentage Change for AL in 2018: {perc_change}")

row_2 = test_df[(test_df['STATE'] == 'AL') & (test_df['YEAR'] == 2017)]
val_2017 = row_2["mean_ADHD"].values[0]

print(f"2017 : {val_2017}, 2018 : {val_2018}")

0     -0.024138
47     0.050725
91    -0.028169
135    0.100775
179   -0.047970
         ...   
134   -0.004367
178    0.040909
221   -0.047619
263    0.004348
307    0.116505
Name: mean_ADHD_perc_change, Length: 300, dtype: float64
Value Change for AL in 2018: -0.3333333400000029
Percentage Change for AL in 2018: -0.01408450732196005
2017 : 23.66666667, 2018 : 23.33333333
